##### Please `pip install -r requirements.txt` first

##### Directory Configuration

* `workDir` points to the Git project, holds the *tsv* file
* `libDir` points to Git project subdirectory hosting python library with files names `go.py  stat_utils.py  uniprot.py`
* `dataDir` points to the data folder with uniprot and GO files

In [2]:
import sys, os
workDir = f"{os.getcwd()}/.."
libDir = f"{workDir}/lib"
dataDir= f"{workDir}/../data" 

sys.path.append(libDir)

In [3]:
import pandas, uniprot, go
import numpy as np

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [6]:
%matplotlib nbagg
import matplotlib.pyplot as plt

## Jeux de données
* Experimental dataset as TSV
* Go ontology as [owl](http://www.obofoundry.org/ontology/go.html)
* Uniprot entries as XML files

## Objectifs
##### 1 Lire et traiter des données d'entrée avec [PANDAS](https://pandas.pydata.org/)

##### 2 Représenter la distribution empirique des "fold-change" (**FC**)
    * Representation d'une grille d'histogrammes
    * Explorer l'effet du paramètre: nombre de catégories
    * Se familliariser avec les objets Axes et Figure
    
##### 3 Evaluer un seuil de **FDR** 5%, avec une approximation par noyau Gaussien de la distribution.
    * Génération d'un jeu de données unimodal (génération par une loi normale)      
    * Determiner un seuil alpha du 5% des valeurs extrêmes dans densité de probabilité


## Ressources
https://www.bioconductor.org/help/course-materials/2015/SeattleApr2015/E_GeneSetEnrichment.html

https://matplotlib.org/gallery.html#




## Format des données expérimentales

Le terme *RA*, désigne le **R**atio d'**A**bondance pour une protéines entre deux experiences.

Les valeurs de **RA** sont stockées dans la colonne ```Log2 Corrected Abundance Ratio```.

Un "ratio" négatif indique qu'une protéine est moins présente dans l'experience considérée par rapport à une experience de référence.

* La condition experimentale de référence est un milieu de croissance bactérienne
* La condition experimentale testée est un milieu riche en Tétracycline 



# Première exploration des données

### Lecture des valeurs experimentales
Charger ```TCL_wt1.tsv``` dans une [dataframe pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html).
* Retirer les lignes présentant des aberrations numériques

##### [Comment filtrer des lignes Pandas](https://cmdlinetips.com/2018/02/how-to-subset-pandas-dataframe-based-on-values-of-a-column/)

In [ ]:
#### A VOUS DE JOUER




#### Représenter la distribution des **RA** des protéines de l'experience [**Log2 Corrected Abundance Ratio**].
Pour cela vous devez, 
##### Réaliser une grille 2x2 avec 4 histogrammes, chacun étant paramétré avec une valeur différente de l'option *bins* .
La méthode [Axes.hist()](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.hist.html#matplotlib.axes.Axes.hist) permet de dessiner des histogrammes.

Rappel: La taile de la grille est donnée par les paramètres *ncols, nrows*, comme dans lci dessous
  
`fig1, ( (ax1, ax2), (ax3, ax4) ) = plt.subplots(figsize=(10,6), ncols=2, nrows=2)`

Les valeurs du paramètre *bin* peuvent fortement influencer la forme de l'histogramme. Choisissez, quatre valeurs illustrant cet effet. Vous ajouterez un titre à chaque histogramme indiquant son nombre de bins/catégories.

In [1]:
#### A VOUS DE JOUER

### Réaliser une Kernel Density Estimation sur un histogramme matplotlib à l'aide du package scipy.
#### 1. En manipulant une représentation déjà produite sur un jeu de données aléatoires
#### 2. En appliquant cette technique au jeu de données expérimentales

###### 1/ Générer 1000 valeurs normalement distribuées.
```python
mu, sigma = 0, 0.1 # mean and standard deviation
s = np.random.normal(mu, sigma, 1000)
print(f"s est un tableau de {len(s)} valeurs normalement distribuées autour de {mu}")
print(f"Les 10 premières valeurs de s sont \n{s[:10]}")
```
<span style="color:green;font-weight:bold">$\uparrow$ Reproduiser cet exemple dans la cellule ci-dessous.$\downarrow$ </span>

###### 2/ Créer une figure dotée d'un seul "axe"
Dessinons-y l'histogramme des valeurs précedemment générées
```python
fig2, f2_ax = plt.subplots(figsize=(5,3), ncols=1, nrows=1)
count, bins, barObj = f2_ax.hist(s, 50, density=True) 
```

<span style="color:green;font-weight:bold">$\uparrow$ Reproduisez cet exemple dans la cellule ci-dessous $\downarrow$ </span>

<span style="color:firebrick;font-weight:bold"> $\rhd$ Inspectez les contenus des variables *count* et *bin* retournées par la méthode Axes.hist()</span>


###### 3/ Estimer la densité de probabilité des 1000 valeurs par noyau Gaussien
Oui c'est bête :D, nous savons que la densité sous-jacente est celle d'une loi normale, mais l'exemple est ainsi assez simple.
```python
from scipy.stats.kde import gaussian_kde
pdf_by_gaussian = gaussian_kde(s)
```

<span style="color:green;font-weight:bold">$\uparrow$ Reproduisez cet exemple dans la cellule ci-dessous $\downarrow$ </span>


###### 3(suite)/ Générer la courbe cette densité
* Prenons 1000 points points également répartis entre le minimum et le maximum du jeu de données
* Générons les valeurs correspondantes dans la densité
```$python
x_kde = np.linspace(min(s),max(s), 1000)
y_kde = pdf_by_gaussian(x_kde)
```
<span style="color:green;font-weight:bold">$\uparrow$ Reproduisez cet exemple dans la cellule ci-dessous $\downarrow$ </span>

<span style="color:firebrick;font-weight:bold"> $\rhd$ Dessiner ces deux listes de valeurs dans un graphique simple </span>

#### 4/ Estimation de densité et histogramme

* Ajoutez sur le même graphique histogramme et courbe de densité

* Mettez le paramètre `density` à False sur `count, bins, barObj = f2_ax.hist(s, 50, density=True)`
    * Qu'observez-vous en ordonnées?
    * Pouvez-vous expliquer le changement de "forme" de la courbe de densité?
    * L'aire sous la courbe et/ou sous l'histogramme change-t-elle avec le booléen density?


* Peut-on imaginer une technique pour dessiner histogramme et courbe de densité avec les bonnes unités en ordonnées ?
    * Les variables count et bins peuvent notamment vous aider à calculer l'aire totale sous l'histogramme.
    
* Dessiner histogramme et courbe de densité avec les bonnes unités et les légendes et titres correspondants.

In [ ]:
#### A VOUS DE JOUER

#### 4/ Recherche du seuil alpha

Le package scipy.integrate permet de calculer l'intégrale exacte d'une fonction 

`integral_value, residu = scipy.integrate.quad(f, borne_inf, borne_sup)`

* A l'aide de l'exemple ci-dessus:

    * Calculer la valeur de l'aire sous la courbe de densité estimée entre les valeurs minimale et maximale du jeux de données
    * Chercher la borne d'intégration inférieure **alpha** correspondant "à peu près" à 5% de l'aire totale sous la courbe de la  de densité de probabilité estimée.


In [ ]:
#### A VOUS DE JOUER

###### 5/ Représentation graphique du seuil alpha
Redessinez le graphique *histogramme et courbe de densité* mais avec:
* une ligne verticale rouge matérialisant le seuil **alpha**
* Un [texte](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.text.html) `alpha(5%) = {valeur de alpha}`
* (Bonus) coloriez en rouge [l'aire sous la courbe](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.fill_between.html) de densité correspondant à l'intégrale calculée.

In [ ]:
#### A VOUS DE JOUER

## Appliquez cette méthode pour determiner un seuil alpha pour le ratio d'abondance
* Liste de valeurs `Log2 Corrected Abundance Ratio`
1. Estimation de la densité par noyau gaussien 
2. Calcul du seuil alpha
3. Représentation graphique
    * Histogramme
    * Densité 
    * seuil alpha

In [ ]:
### Cellule Exercice